In [67]:
import pandas as pd
import plotly.express as px
import numpy as np

In [68]:
datos = pd.read_csv("BoyacaIcfes.csv")
datos.shape

(215234, 51)

In [69]:
datos.isnull().sum()

periodo                              0
estu_tipodocumento                   0
estu_consecutivo                     0
cole_area_ubicacion                 80
cole_bilingue                    17277
cole_calendario                     49
cole_caracter                      980
cole_cod_dane_establecimiento        0
cole_cod_dane_sede                   0
cole_cod_depto_ubicacion             0
cole_cod_mcpio_ubicacion             0
cole_codigo_icfes                 1616
cole_depto_ubicacion                 0
cole_genero                          0
cole_jornada                         0
cole_mcpio_ubicacion                 0
cole_naturaleza                      0
cole_nombre_establecimiento          0
cole_nombre_sede                     0
cole_sede_principal                  0
estu_cod_depto_presentacion          0
estu_cod_mcpio_presentacion          0
estu_cod_reside_depto             1234
estu_cod_reside_mcpio             1234
estu_depto_presentacion              0
estu_depto_reside        

In [70]:
# se realiza un intento de recuperar información recuperable
datos["cole_bilingue"] = datos.groupby("cole_cod_dane_sede")["cole_bilingue"].bfill()
datos["cole_caracter"] = datos.groupby("cole_cod_dane_sede")["cole_caracter"].bfill()

In [71]:
#Se elimina el resto de entradas debido a que no es información recuperable
datos.dropna(inplace=True)
datos.shape

(114689, 51)

In [72]:
#Se eliminan filas duplicadas
datos.drop_duplicates(inplace=True)
datos.shape

(87898, 51)

In [73]:
datos["periodo"] = datos["periodo"].astype(str)
datos["año"] = datos["periodo"].str[:4]
datos["semestre"] = datos["periodo"].str[4]
datos["año"] = datos["año"].astype(int)
datos["semestre"] = datos["semestre"].astype(int)
datos.drop("periodo", axis =1, inplace=True)

In [74]:
date_format = '%d/%m/%Y'
# Convertir las cadenas a fechas
datos['estu_fechanacimiento'] = pd.to_datetime(datos['estu_fechanacimiento'], format=date_format, errors='coerce')
datos.loc[(datos['estu_fechanacimiento'] < pd.to_datetime('1924-01-01')) | (datos['estu_fechanacimiento'] > pd.to_datetime('2022-12-31')), 'estu_fechanacimiento'] = pd.NaT

In [75]:
datos.dropna(inplace=True)
datos.shape

(87884, 52)

In [76]:
#Remover las columnas con una unica entrada
datos.drop(["cole_cod_depto_ubicacion", "cole_depto_ubicacion","estu_estudiante", "cole_calendario"], axis = 1, inplace = True)
#Remover las columnas que contienen la misma información
datos.drop(["cole_mcpio_ubicacion", "estu_depto_presentacion","estu_depto_reside","estu_mcpio_presentacion", "estu_mcpio_reside"], axis = 1, inplace = True)
#Remover las columnas que contienen la misma información
datos.drop(["cole_codigo_icfes","cole_sede_principal","cole_nombre_establecimiento", "cole_nombre_sede"], axis = 1, inplace = True)

In [77]:
#Remover las columnas iguales
if datos['estu_nacionalidad'].equals(datos['estu_pais_reside']):
    datos.drop(['estu_pais_reside'], axis = 1, inplace = True)

In [78]:
# limpiando la entrada de personas en hogar
def agrupar_personas(x):
    if x in ['1 a 2', 'Uno', 'Dos']:
        return '1 a 2'
    elif x in ['3 a 4', 'Tres', 'Cuatro']:
        return '3 a 4'
    elif x in ['5 a 6', 'Cinco', 'Seis']:
        return '5 a 6'
    elif x in ['7 a 8', 'Siete', 'Ocho']:
        return '7 a 8'
    elif x in ['9 o más', 'Nueve', 'Diez', 'Once', 'Doce o más']:
        return '9 o más'
    else:
        return 'nan'

datos['fami_personashogar'] = datos['fami_personashogar'].apply(agrupar_personas)

In [79]:
# limpiando la entrada de personas en hogar
def agrupar_cuartos(x):
    if x in ['Seis', 'Siete', "Ocho", "Nueve", "Diez o más"]:
        return "Seis o mas"

datos['fami_cuartoshogar'] = datos['fami_cuartoshogar'].apply(agrupar_personas)

In [80]:
datos.head()  

,estu_tipodocumento,estu_consecutivo,cole_area_ubicacion,cole_bilingue,cole_caracter,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_cod_mcpio_ubicacion,cole_genero,cole_jornada,...,fami_tienelavadora,desemp_ingles,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global,año,semestre
0,TI,SB11201520132216,URBANO,N,TÉCNICO/ACADÉMICO,115047000019,115047000019,15047,MIXTO,COMPLETA,...,Si,A1,53.0,62.0,53.0,51.0,54.0,274.0,2015,2
3,TI,SB11201420197609,URBANO,N,TÉCNICO/ACADÉMICO,115759000279,115759000279,15759,MIXTO,MAÑANA,...,Si,A2,68.0,72.0,69.0,66.0,72.0,348.0,2014,2
4,TI,SB11201620449437,URBANO,N,ACADÉMICO,115238000728,115238000728,15238,MIXTO,MAÑANA,...,Si,A1,49.0,56.0,49.0,51.0,60.0,268.0,2016,2
5,TI,SB11201620046451,URBANO,N,TÉCNICO,115212000266,115212000266,15212,MIXTO,COMPLETA,...,No,A1,52.0,68.0,52.0,54.0,64.0,295.0,2016,2
6,TI,SB11201940129372,RURAL,N,TÉCNICO/ACADÉMICO,215204000294,215204000294,15204,MIXTO,MAÑANA,...,Si,A-,40.0,59.0,28.0,45.0,51.0,227.0,2019,4


In [81]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87884 entries, 0 to 215233
Data columns (total 38 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   estu_tipodocumento             87884 non-null  object        
 1   estu_consecutivo               87884 non-null  object        
 2   cole_area_ubicacion            87884 non-null  object        
 3   cole_bilingue                  87884 non-null  object        
 4   cole_caracter                  87884 non-null  object        
 5   cole_cod_dane_establecimiento  87884 non-null  int64         
 6   cole_cod_dane_sede             87884 non-null  int64         
 7   cole_cod_mcpio_ubicacion       87884 non-null  int64         
 8   cole_genero                    87884 non-null  object        
 9   cole_jornada                   87884 non-null  object        
 10  cole_naturaleza                87884 non-null  object        
 11  estu_cod_depto_pres

In [82]:
datos.describe()

,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_cod_mcpio_ubicacion,estu_cod_depto_presentacion,estu_cod_mcpio_presentacion,estu_cod_reside_depto,estu_cod_reside_mcpio,estu_fechanacimiento,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global,año,semestre
count,8.788400e+04,8.788400e+04,87884.000000,87884.000000,87884.000000,87884.000000,87884.000000,87884,87884.000000,87884.000000,87884.000000,87884.000000,87884.000000,87884.000000,87884.000000,87884.000000
mean,1.847148e+11,1.869382e+11,15416.442435,15.754586,16124.316281,15.093680,15504.619578,2000-01-01 15:07:39.605844224,51.641892,54.182240,51.823643,53.243844,53.414945,265.245153,2016.968527,2.599574
min,1.150010e+11,1.150010e+11,15001.000000,5.000000,5001.000000,5.000000,5001.000000,1943-08-15 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,2014.000000,1.000000
25%,1.152990e+11,1.152990e+11,15176.000000,15.000000,15176.000000,15.000000,15176.000000,1998-03-14 00:00:00,44.000000,47.000000,44.000000,47.000000,47.000000,232.000000,2015.000000,2.000000
50%,1.157590e+11,1.157590e+11,15407.000000,15.000000,15299.000000,15.000000,15407.000000,1999-09-20 00:00:00,50.000000,54.000000,52.000000,53.000000,53.000000,263.000000,2016.000000,2.000000
75%,3.150010e+11,3.150010e+11,15740.000000,15.000000,15757.000000,15.000000,15740.000000,2002-02-03 00:00:00,58.000000,62.000000,59.000000,60.000000,60.000000,296.000000,2019.000000,4.000000
max,4.158390e+11,5.154690e+11,15897.000000,99.000000,99001.000000,85.000000,85001.000000,2019-05-05 00:00:00,100.000000,100.000000,100.000000,100.000000,100.000000,471.000000,2022.000000,4.000000
std,8.964484e+10,9.105001e+10,286.372922,6.632418,6629.134376,2.051302,2071.452325,NaN,10.569924,11.224778,10.864054,9.519391,9.646706,45.550375,2.589180,0.924771


In [83]:
datos.to_csv("IcfesLimpio.csv", index=False)